In [7]:
pip install googletrans==3.1.0a0 --user

  Using cached googletrans-3.1.0a0-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.
spyder 4.2.5 requires jedi==0.17.2, but you have jedi 0.18.2 which is incompatible.
spyder 4.2.5 requires parso==0.7.0, but you have parso 0.8.3 which is incompatible.


In [2]:
import nucleus_api
import os
import csv
import json
import datetime
import time
from nucleus_api.rest import ApiException
import nucleus_api.api.nucleus_api as nucleus_helper
from pprint import pprint
import numpy as np
from pathlib import Path
from pymed import PubMed

configuration = nucleus_api.Configuration()
# configuration.host = 'localhost:5000'
configuration.host = 'http://3.91.117.194:5000/'
configuration.api_key['x-api-key'] = 'YTRgtK8T2H-hhR5NbmKW3w'

api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [3]:
dataset = 'PubMedDataset'
#create dataset

In [12]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def hello():
    return "pikachu"

@app.route('/home')
def home():
    # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
    query = ''
    custom_stop_words = [] # str | List of stop words. (optional)
    num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
    metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
    #time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
    remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

    #compute topics:
    payload = nucleus_api.Topics(dataset=dataset,                                
                                    query=query,                   
                                    custom_stop_words=custom_stop_words,     
                                    num_topics=num_topics,
                                    metadata_selection=metadata_selection,
                                    remove_redundancies=remove_redundancies)
        
    api_response = api_instance.post_topic_api(payload) 
    input_array = [["Topic", "Strength"]] + [[topic.keywords, float(topic.strength)] for topic in api_response.result.topics]
    return render_template('home.html', inputArray = input_array)

if __name__ == '__main__':
    app.run(host = "0.0.0.0", port=5002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:5002/ (Press CTRL+C to quit)
172.17.0.1 - - [26/Feb/2023 18:23:08] "GET /home HTTP/1.1" 200 -


In [2]:
try:
    ip = get_ipython()
    running_notebook = True
except NameError:
    running_notebook = False
print(running_notebook)

True


In [49]:


pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(Mussels[Title]) AND (10.1155/2022/8066356[DOI])", max_results=5)
for article in results:
    print (article.toJSON())

{
    "abstract": "The aim of the present study was to evaluate hepatitis A virus (HAV) and hepatitis E virus (HEV) contamination in mussels (",
    "authors": [
        {
            "affiliation": "Team of Virology Oncology and Biotechnology, Laboratory of Virology Oncology Biosciences Environment and New Energies, Faculty of Sciences & Technologies Mohammedia, University Hassan II of Casablanca, Casablanca, Morocco.",
            "firstname": "Hana\u00e2",
            "initials": "H",
            "lastname": "Bazir"
        },
        {
            "affiliation": "Team of Virology Oncology and Biotechnology, Laboratory of Virology Oncology Biosciences Environment and New Energies, Faculty of Sciences & Technologies Mohammedia, University Hassan II of Casablanca, Casablanca, Morocco.",
            "firstname": "Najwa",
            "initials": "N",
            "lastname": "Hassou"
        },
        {
            "affiliation": "Casablanca Regional Research and Analysis Laboratory of 

In [5]:
dataset = 'PubMedDataset'
#create dataset

In [6]:

pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(e[Title]) AND ([DOI])", max_results=2000)
for article in results:
    newdocument = {}
    newdocument["title"] = article.title
    if type(article.publication_date) == str:
        newdocument["time"] = article.publication_date
    else:
        newdocument["time"] = article.publication_date.strftime("%Y-%m-%d")
    newdocument["content"] = article.abstract
    # load into dataset
    payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                       document=newdocument)
    try:
        api_response = api_instance.post_append_json_to_dataset(payload)
        print(api_response.result)
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])


{'n_documents': 1750, 'size': 1208}
{'n_documents': 1751, 'size': 21}
{'n_documents': 1752, 'size': 1363}
{'n_documents': 1753, 'size': 1382}
{'n_documents': 1754, 'size': 255}
{'n_documents': 1755, 'size': 1975}
{'n_documents': 1756, 'size': 21}
{'n_documents': 1757, 'size': 47}
{'n_documents': 1758, 'size': 80}
{'n_documents': 1759, 'size': 1023}
{'n_documents': 1760, 'size': 121}
{'n_documents': 1761, 'size': 1374}
ERROR: {'message': 'document must have nonempty content and title'}
{'n_documents': 1762, 'size': 66}
{'n_documents': 1763, 'size': 1303}
{'n_documents': 1764, 'size': 762}
{'n_documents': 1765, 'size': 1438}
{'n_documents': 1766, 'size': 2117}
{'n_documents': 1767, 'size': 1198}
{'n_documents': 1768, 'size': 1087}
{'n_documents': 1769, 'size': 1361}
{'n_documents': 1770, 'size': 1747}
{'n_documents': 1771, 'size': 360}
{'n_documents': 1772, 'size': 1084}
{'n_documents': 1773, 'size': 1485}
{'n_documents': 1774, 'size': 2343}
{'n_documents': 1775, 'size': 2084}
{'n_docume

KeyboardInterrupt: 

In [3]:
# str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
#time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

#compute topics:
payload = nucleus_api.Topics(dataset=dataset,                                
                                 query=query,                   
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies)
    
api_response = api_instance.post_topic_api(payload) 
#create a json file topics.json containing only topic keywords, weights and strength
topics = [{'keywords':topic.keywords, 
           'keywords_weight':topic.keywords_weight, 
           'strength':topic.strength} for topic in api_response.result.topics]
# print(json.dumps(topics, indent=4, ensure_ascii=False))
input_array = [["Topic", "Strength"]] + [[topic.keywords, topic.strength] for topic in api_response.result.topics]
# with open('demo_topics.json', 'w') as outfile:
# with open('/demo/demo_topics.json', 'w') as outfile:
#     json.dump(topics, outfile, indent=4, ensure_ascii=False
print(input_array.__str__())

[['Topic', 'Strength'], ['hepatitis virus;virus hev;hev infection;public health;viral hepatitis;acute viral;acute hepatitis;common acute', '0.2527'], ['covid pandemic;study aimed;nursing students;social media;health promoting;promoting behaviors;aimed investigate;learning covid', '0.3166'], ['cigarette users;current cigarette;users cigarette;non smokers;compared non;non users;tobacco cigarette;quit vaping', '0.2135'], ['cross sectional;sectional study;sectional survey;study conducted;survey conducted;sectional descriptive;conducted cross;descriptive cross', '0.1893'], ['level cigarette;low level;depressive symptoms;symptoms aces;greater depressive;cigarette greater;high level;stable low;cannabis stable;age cannabis;aces extraversion', '0.0277']]


In [6]:
topic_keywords = [topic[0].split(";") for topic in input_array]
print(topic_keywords)

[['Topic'], ['hepatitis virus', 'virus hev', 'hev infection', 'public health', 'viral hepatitis', 'acute viral', 'acute hepatitis', 'common acute'], ['covid pandemic', 'study aimed', 'nursing students', 'social media', 'health promoting', 'promoting behaviors', 'aimed investigate', 'learning covid'], ['cigarette users', 'current cigarette', 'users cigarette', 'non smokers', 'compared non', 'non users', 'tobacco cigarette', 'quit vaping'], ['cross sectional', 'sectional study', 'sectional survey', 'study conducted', 'survey conducted', 'sectional descriptive', 'conducted cross', 'descriptive cross'], ['level cigarette', 'low level', 'depressive symptoms', 'symptoms aces', 'greater depressive', 'cigarette greater', 'high level', 'stable low', 'cannabis stable', 'age cannabis', 'aces extraversion']]


In [8]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
for topic in topic_summarization:
  translations = translator.translate(topic, dest='vi')
  topic_translated = [translation.text for translation in translations]
  print(topic_translated)


['Đề tài']
['virus viêm gan', 'virus với nhau', 'nhiễm trùng hev', 'sức khỏe cộng đồng', 'viêm gan siêu vi', 'virus cấp tính', 'viêm gan cấp', 'cấp tính thông thường']
['đại dịch covid', 'nghiên cứu nhằm mục đích', 'sinh viên điều dưỡng', 'truyền thông xã hội', 'tăng cường sức khỏe', 'thúc đẩy hành vi', 'nhằm mục đích điều tra', 'học covid']
['người sử dụng thuốc lá', 'thuốc lá hiện tại', 'người dùng thuốc lá', 'người không hút thuốc', 'so khong', 'người dùng không', 'thuốc lá thuốc lá', 'bỏ thuốc lá điện tử']
['mặt cắt ngang', 'nghiên cứu từng phần', 'khảo sát mặt cắt', 'nghiên cứu tiến hành', 'khảo sát tiến hành', 'mô tả mặt cắt', 'tiến hành chéo', 'chéo mô tả']
['cấp thuốc lá', 'cấp thấp', 'triệu chứng trầm cảm', 'triệu chứng', 'trầm cảm hơn', 'thuốc lá lớn hơn', 'cấp độ cao', 'thấp ổn định', 'cần sa ổn định', 'tuổi cần sa', 'át chủ bài']


In [42]:
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 4 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
summary_length = 3 # int | The maximum number of bullet points a user wants to see in each topic summary. (optional) (default to 6)
context_amount = 0 # int | The number of sentences surrounding key summary sentences in the documents that they come from. (optional) (default to 0)
num_docs = 20 # int | The maximum number of key documents to use for summarization. (optional) (default to 20)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)
metadata_selection = ""
try:
    payload = nucleus_api.TopicSummaryModel	(dataset=dataset,                                
                                 query=query,
                                 summary_length = summary_length,
                                 num_keywords = 8,                 
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies,
                                 time_period = "5Y"
)
    api_response = api_instance.post_topic_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topic_summarization = []
    for i,res in enumerate(api_response.result):
        print('Topic', i, 'summary:')
        print('    Keywords:', res.keywords)
        summarization = ""
        for j in range(len(res.summary)):
            print(res.summary[j])
            summarization += res.summary[j].sentences[2:-2]
        print(summarization)
        topic_summarization.append(summarization)
        print('---------------')
print(topic_summarization)

INFO: Start polling job status of 7515
INFO: Job 7515 completed.
Topic 0 summary:
    Keywords: hepatitis virus;virus hev;hev infection;public health;viral hepatitis;acute viral;virus hav;health concern
{'attribute': {'counts': None, 'time': 1673308800},
 'doc_id': '8443559087698784061',
 'sentences': "['Hepatitis E virus (HEV) infection is a significant public "
              'health problem, which infects 20 million individuals every '
              "year.']",
 'title': 'Acute Kidney Injury as a Rare Complication of Acute Hepatitis E in '
          'a Child; A Case Report.'}
{'attribute': {'counts': None, 'time': 1671753600},
 'doc_id': '1628369764488631278',
 'sentences': "['Hepatitis\\xa0E virus (HEV) infection can cause severe viral "
              "hepatitis and eventually liver failure.']",
 'title': 'Low mortality despite temporary liver dysfunction in severe courses '
          'of acute hepatitis\xa0E.'}
{'attribute': {'counts': None, 'time': 1670544000},
 'doc_id': '84037472

In [37]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
topic_translated = []
for topic in topic_summarization:
  translation = translator.translate(topic, dest='vi')
  topic_translated += [translation.text]
  print(translation.text)

Các biện pháp lặp đi lặp lại phân tích hồ sơ tiềm ẩn (RMLPAs) đã kiểm tra quỹ đạo thuốc lá và thuốc lá điện tử trong 6 tháng qua trên 5 đợt dữ liệu (2018-2020) trong số 3.006 thanh niên (MRMLPA mang lại giải pháp 6 hồ sơ, được liên kết duy nhất với các yếu tố dự báo: người sử dụng thuốc lá và thuốc lá điện tử ở mức độ thấp hoặc không ổn định (66,3%; nhóm tham khảo), sử dụng thuốc lá và thuốc lá điện tử ở mức độ thấp ổn định (12,3%; các triệu chứng trầm cảm nặng hơn, ACE, cởi mở; nam, Da trắng, cần sa mức độ trung bình ổn định và sử dụng thuốc lá điện tử mức độ thấp (6,2%; các triệu chứng trầm cảm, ACE, hướng ngoại nhiều hơn; ít cởi mở, tận tâm hơn; tuổi lớn hơn, nam giới, Da đen hoặc gốc Tây Ban Nha, sử dụng cần sa), mức độ thấp ổn định thuốc lá và giảm sử dụng thuốc lá điện tử (6,0%; triệu chứng trầm cảm nhiều hơn, ACE, cởi mở; tuổi trẻ hơn, sử dụng cần sa), sử dụng thuốc lá điện tử ở mức độ cao và mức độ ổn định (4,7%; triệu chứng trầm cảm, ACE, hướng ngoại nhiều hơn) ; lớn tuổi hơn,

In [59]:
# update_period = 'd' # str | Frequency at which the historical anlaysis is performed. choices=["d","m","H","M"] (default to d)
# query = '' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
# custom_stop_words = [] # str | List of stop words (optional)
# num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
# #num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
# inc_step = 1 # int | Number of increments of the udpate period in between two historical computations. (optional) (default to 1)
# excluded_docs = [''] # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
# custom_dict_file = {} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)
# remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "2020-01-01 01:01:01" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "2024-01-01 01:01:01" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
api_response = None

# str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
#time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)


try:
    payload = nucleus_api.TopicHistoryModel(dataset=dataset,                                
                                 query=query,                   
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies,
                                 time_period = "1M"
                                 )
    api_response = api_instance.post_topic_historical_analysis_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    print(e)

print('Printing historical metrics data...')

for i,res in enumerate(api_response.result):
    print('Topic', i, res.keywords)
    print('    Timestamps:', res.time_stamps)
    print('    Strengths:', res.strengths)
    print('    Consensuses:', res.consensuses)
    print('    Sentiments:', res.sentiments)
    print('----------------')
            

Printing historical metrics data...
Topic 0 cross sectional;sectional study;study aimed;study conducted;aimed investigate;covid pandemic;nursing students;study carried
    Timestamps: ['2023-02-02 00:00:00', '2023-02-05 00:00:00', '2023-02-11 00:00:00', '2023-02-12 00:00:00', '2023-02-16 00:00:00', '2023-02-23 00:00:00', '2023-02-24 00:00:00', '2023-02-26 00:00:00']
    Strengths: ['0.2955', '0.2533', '0.3343', '0.4425', '0.4206', '0.4085', '0.3481', '0.3196']
    Consensuses: ['0.5164', '0.6524', '0.6388', '0.4524', '0.4581', '0.5271', '0.852', '0.7383']
    Sentiments: ['0.0039', '-0.03', '-0.0612', '-0.0019', '-0.0226', '-0.045', '-0.0976', '-0.0894']
----------------
Topic 1 cigarette users;tobacco cigarette;users cigarette;non smokers;dual users;users dual;smokers tobacco;users tobacco
    Timestamps: ['2023-02-02 00:00:00', '2023-02-05 00:00:00', '2023-02-11 00:00:00', '2023-02-12 00:00:00', '2023-02-16 00:00:00', '2023-02-23 00:00:00', '2023-02-24 00:00:00', '2023-02-26 00:00:00